* Filter:
    - Importance: High => Low
    - Medium questions only

## Flatten

* Clarification questions:
    - What type of data does the array contain? Some approach only applies to certain data types
    - How many levels of nesting can this array have? If there are thousands-of-levels of nesting, recursion might not be a good idea given its big upfront memory footprint.
    - Should we return a new array or we mutate the existing array?
        * would use splice
    - Can we assume valid input, i.e. an array. Normally the answer is "yes", so you don't have to waste your time doing defensive programming.
    - Does the environment the code runs on has ES6+ support? The environment determines what methods/native APIs you have access to.


In [ ]:
/**
 * @param {Array<*|Array>} value
 * @return {Array}
 * 
 * returns a new array
 * 
 * no matter how deep, we want to traverse it
 * definitely recursive
 *  - if we encounter an array, we traverse it
 *  - definitely have to check for sparsity
 */

// recursive
export default function flatten(value) {
  const result = [];

  const traverse = (arr) => {
    for (const element of arr) {
      if (Array.isArray(element)) {
        traverse(element);
      } else {
        result.push(element);
      }
    }
  }

  traverse(value);
  return result;
}

/*
 * iterative version
 * we use a stack instead
 * and reverse it at the end
 * so this is only around O(n) time complexity
*/
export default function flatten(value) {
  const res = [];
  const copy = [...value];

  while (copy.length) {
    const item = copy.pop();
    if (Array.isArray(item)) {
      copy.push(...item);
    } else {
      res.push(item);
    }
  }

  return res.reverse();
}

## Promise.all()

* use `forEach` so that you can pass in an async function
    - this allows you to await on the element concurrently on all items since `forEach` is synchronous
* new Array(arrLen) is much faster than Array.from({ length: arrLen })
* you don't need a while loop or anything to wait for counter to become 0
    - you just need the last item to be resolved to check if counter = 0, then you can resolve the entire result

In [ ]:
/**
 * @param {Array} iterable
 * @return {Promise<Array>}
 * 
 * returns a single promise that resolves to an array of results from input
 * 
 * that promise will only resolve when all of the input's promises have resolved
 * or if input iterable has no promiseshttp://localhost:8888/notebooks/OneDrive/Desktop/Great-Front-End/JavaScript-Polyfills/Medium.ipynb#
 * 
 * rejects immediately if any of the inputs reject or throw an error
 * will reject with first rejection message/error
 * 
 * will probably have to use .then() for error handling
 * 
 */
export default function promiseAll(iterable) {
  // how do we know if all of them resolved?
  // will probably need a counter equal to # of iterables
  // if a value resolves right away or is not a promise, we counter--
  // if a promise resolves, we counter-- in .then() block
  // if we have a an error, we reject immediately

  // how do we check if all of them are done?
  // could have a while loop that continually checks if counter === 0
  // and then we resolve the resulting array
  return new Promise((resolve, reject) => {
    let counter = iterable.length;
    const result = new Array(counter);
    if (counter === 0) {
      resolve(result);
      return;
    }

    iterable.forEach(async(element, i) => {
      try {
        const value = await element;
        result[i] = value;
        counter--;
        if (counter === 0) {
          resolve(result);
        }
      } catch(e) {
        reject(e);
      }
    });
  });
}

## Array.prototype.concat()

* pretty straightforward
* just have to remember that anytime you're dealing with prototype method implementations, you want to use `this` to access the object
* `Array.isArray()` is extremely useful when you might have to handle nested arrays
* `spread syntax (...)` is an invaluable tool

In [ ]:
/**
 * @template T
 * @param {...(T | Array<T>)} items
 * @return {Array<T>}
 * 
 * does not mutate arrays, returns NEW array
 * 
 * if passed in as array, must spread them into result
 *  - what about nested arrays?
 *  - yes, must also handle nested arrays but by how much????
 *  - only the top level
 *  - e.g. [1, 2, 3].concat(4, [5, 6]) => [1, 2, 3, 4, 5, 6]
 *  - [1, 2, 3].concat(4, [[5], 6]) => [1, 2, 3, 4, [5], 6]
 * 
 * if passed in as list of arguments, must also push them into new array
 * 
 * must also handle sparse arrays
 *  - probably want to push undefined
 */
Array.prototype.myConcat = function (...items) {
  const result = [...this];
  // items = array of arguments

  // loop through each item
  for (const item of items) {
    // if item is an array, use spread operator to and push into result
    if (Array.isArray(item)) {
      result.push(...item);
    } else {
      result.push(item);
    }
  }

  // else, just push into result

  // return new array
  return result;
};

## Event Emitter

* Clarification questions
    - The following are good questions to ask the interviewer to demonstrate your thoughtfulness. Depending on their response, you might need to adjust the implementation accordingly.
    - Can emitter.emit() be called without any arguments besides the eventName?
        * Yes, it can be.
        * spreading an empty args[] into function call handles this
    - Can the same listener be added multiple times with the same eventName?
        * Yes, it can be. It will be called once for each time it is added when eventName is emitted in the order they were added.
        * pushing callbacks into listeners array handles this
    - Following up on the question above, what should happen if a listener is added multiple times and emitter.off() is being called once for that listener?
        * The listener will only be removed once.
        * using `indexOf` already handles this since indexOf only returns the first instance of the object it finds
    - Can non-existent events be emitted?
        * Yes, but nothing should happen and the code should not error or crash.
        * we return early for these anyways
    - What should the this value of the listeners be?
        * It can be null.
        * we don't use `.call()` or `.apply()` so it should be null
        * however, if the `this` value can be set, using those 2 methods and passing the thisArg into it would be appropriate
    - Can listeners contain code that invoke methods on the emitter instance?
        * Yes, but we can ignore that scenario for this question.
        * this would probably only work if the callback passed in the EventEmitter instance as a thisArg
            - this would mean we would have to use `.call()` or `.apply()`
    - What if the listener callbacks throw an error during emitter.emit()?
        * The error should be caught and not halt the rest of the execution. However, we will not test for this case.
        * probably just have a try/catch or something, not too sure
* We will handle all the above cases except for the last two cases.
***
* __using `Maps` is a VERY good way of making sure that eventNames don't clash with built-in properties__
    - if you can't use a map, using `Object.create(null)` which creates an object with no prototype and no properties
* return `this` if you want to return the instance of the class
* `.splice(index, # of item to delete)` is very useful for deleting an item from an array
    - `.splice(2, 1)` means we delete 1 item at index 2
    - if you provide a third argument, it will instead insert the third argument(s) at that position

In [ ]:
// You are free to use alternative approaches of
// instantiating the EventEmitter as long as the
// default export has the same interface.

export default class EventEmitter {
  #events = new Map();

  /**
   * @param {string} eventName
   * @param {Function} listener
   * @returns {EventEmitter}
   * 
   * add listener to array of listeners
   * 
   * {
   *  'myEvent': [listener1, listener2, listener3],
   *  'myEvent 2': [listener1,]
   * }
   */
  on(eventName, listener) {
    if (!this.#events.has(eventName)) {
      this.#events.set(eventName, []);
    }
    this.#events.get(eventName).push(listener);
    return this;
  }

  /**
   * @param {string} eventName
   * @param {Function} listener
   * @returns {EventEmitter}
   * 
   * i cannot uniquely identify anonymous functions
   * we can only compare references of the functions
   * probably easier if they are named
   * 
   * indexOf, then splice it out!
   */
  off(eventName, listener) {
    if (!this.#events.has(eventName)) return this;
    const listeners = this.#events.get(eventName);
    const index = listeners.indexOf(listener);
    listeners.splice(index, 1);
    return this;
  }

  /**
   * @param {string} eventName
   * @param  {...any} args
   * @returns {boolean}
   * 
   * for each listener from event, call its function
   */
  emit(eventName, ...args) {
    if (!this.#events.has(eventName)) return false;

    const listeners = this.#events.get(eventName);
    if (listeners.length === 0) return false;

    // clones listeners arr in case one of the callbacks mutates it
    [...listeners].forEach(listener => {
      listener(...args);
    });

    return true;
  }
}

## Event Emitter II

* Clarification questions
    - The following are good questions to ask the interviewer to demonstrate your thoughtfulness. Depending on their response, you might need to adjust the implementation accordingly.
    - Can emitter.emit() be called without any arguments besides the eventName?
        * Yes, it can be.
    - Can the same listener be added multiple times with the same eventName?
        * Yes, it can be. It will be called once for each time it is added when eventName is emitted in the order they were added.
    - Can non-existent events be emitted?
        * Yes, but nothing should happen and the code should not error or crash.
    - What should the this value of the listeners be?
        * It can be null.
    - Can sub.off() be called more than once?
        * Yes it can be, the second call should be a no-op.
    - Can listeners contain code that invoke methods on the emitter instance?
        * Yes, but we can ignore that scenario for this question.
    - What if the listener callbacks throw an error during emitter.emit()?
        * The error should be caught and not halt the rest of the execution. However, we will not test for this case.
* We will handle all the above cases except for the last two cases.
***
* same as Event Emitter I but you can take advantage of closures
* you can create a plain object and create a named method called 'off' to return to the user to call whenever they want to remove the listener
    - if you use the `this` keyword, you must bind your `this` to the object
    - or you can just take advantage of closures and create a constant whose value is equal to `this` and use that inside the 'off' method

In [ ]:
// You are free to use alternative approaches of
// instantiating the EventEmitter as long as the
// default export is correct.
export default class EventEmitter {
  #events = new Map();

  /**
   * @param {string} eventName
   * @param {Function} listener
   * @returns {{off: Function}}
   * 
   * you probably just want to grab the index of the callback
   * inside its listeners array
   * 
   * then when you call off(), you just remove it
   * might want to bind this to function returned by off
   * {
   *  off: func(){}
   * }.bind(this);
   */
  on(eventName, listener) {
    if (!this.#events.has(eventName)) {
      this.#events.set(eventName, []);
    }
    this.#events.get(eventName).push(listener);
    return {
      off: function() {
        const listeners = this.#events.get(eventName);
        const index = listeners.indexOf(listener);
        listeners.splice(index, 1);
      }.bind(this)
    }
  }

  /**
   * @param {string} eventName
   * @param {...any} args
   * @returns boolean
   */
  emit(eventName, ...args) {
    if (!this.#events.has(eventName)) return false;
    const listeners = this.#events.get(eventName);
    if (listeners.length === 0) return false;

    [...listeners].forEach(listener => {
      listener(...args);
    });
    return true;
  }
}

## getElementsByClassName

* __always remember to be cognizant of the input and output__
    - I mistakenly returned list of strings instead of the element itself
* `Element.classList.contains()` is extremely useful for finding if a classList contains a class name
    - Element.classList preferred over Element.className because element.className is a string
    - `Element.classList.forEach()` is available
    - Element.classList is a __live__ `DOMTokenList`
* `Element.children` is preffered over `Element.childNodes` because __a node could literally be non-elements like text nodes__
    - both of these are __live__!
    - `Element.children` is an `HTMLCollection` so you must either use spread syntax (`...`) or use `for...of` to iterate over it
* keep in mind, the element passed in is the doc and you only want to look at its descendants!!!
* `Array.prototype.every()` is very useful for making sure that every item in Array matches a condition passed into it
    - e.g. [1, 2, 3].every(item => item < 4) === true

In [ ]:
/**
 * @param {Element} element
 * @param {string} classNames
 * @return {Array<Element>}
 * 
 * * returns list of DESCENDANT elements which has the specified className
 * * classNames = string of class names separated by white-space
 * * only descendants searched, not element itself
 * 
 * element is made up of:
 * [element-type].[all.of.element's.classes.separated.by.dots]
 * e.g. classString = 'foo bar'
 * <div class="foo bar baz"></div> => matches
 *  - its result = div.foo.bar.baz
 * <div class="foo bar"></div> => matches
 *  - its result = div.foo.bar
 * 
 *  the element that is passed in is the doc
 *  - you want to look at its children
 * 
 * doc.body.children[0].classList.forEach(className => {
  console.log({ className });
})
 */

const extractClasses = (classNames) => {
  const res = [];
  let current = '';
  for (let i = 0; i < classNames.length; i++) {
    if (classNames[i] === " ") {
      if (current) res.push(current);
      current = '';
      continue;
    }
    current += classNames[i];
  }
  if (current) res.push(current);
  return res;
}

const containsClasses = (child, classNamesArr) => {
  // // check children's classList against classNames
  return classNamesArr.every(cname => child.classList.contains(cname));
}

export default function getElementsByClassName(element, classNames) {
  // classNames = string and we want fast look up
  // can create a map of classNames by separating them by white-space
  const classNamesArr = extractClasses(classNames);
  const result = [];

  const traverse = (node) => {
    // want to iterate through all children
    const children = node?.children;
    if (!children) return;

    for (const child of children) {
      // if element has all of it, add it to result array
      if (containsClasses(child, classNamesArr)) {
        result.push(child);
      }
      traverse(child);
    }
  }

  traverse(element);
  return result;
}

## Promise.allSettled()

* pretty much the same as Promise.allSettled()
* `.forEach()` can run all promises in parallel
    - just await on item and wrap in try/catch

In [ ]:
/**
 * @param {Array} iterable
 * @return {Promise<Array<{status: 'fulfilled', value: *}|{status: 'rejected', reason: *}>>}
 * 
 * returns a promise that resolves after all promises in iterable are fulfilled or rejected
 * an array of objects:
 * {
 * status: 'fulfilled',
 * value: 'some value returned from fulfillment'
 * }
 * or 
 * {
 * status: 'rejected',
 * reason: 'reason for rejection',
 * }
 * 
 * if iterable is empty, return a promise object with an empty array
 */
export default function promiseAllSettled(iterable) {
  // if iterable is empty, resolve with empty array
  if (iterable.length === 0) {
    return Promise.resolve([]);
  }

  return new Promise(resolve => {
    const result = new Array(iterable.length);
    let counter = 0; // counts # of settled promises
    // iterate through iterable and run them in parallel
    iterable.forEach(async (item, index) => {
      try {
        const value = await item;
        result[index] = {
          status: 'fulfilled',
          value,
        };
      } catch(err) {
        result[index] = {
          status: 'rejected',
          reason: err,
        };
      }
      counter++; // increments as long as item was settled

      // if # of settled items === # of items in iterable
      // we can resolve the resulting array
      if (counter === iterable.length) {
        resolve(result);
      }
    });
  })
}

## getElementsByTagName

* `Element.tagName` returns an UPPERCASE tagName
    - definitely just want to convert the tagName argument into its upper case version since this operation happens once instead of converting to lower case for every element we see
* `Element.children` is a live HTMLCollection that does not have .forEach available so you must use `for...of` to iterate over its items

In [ ]:
/**
 * @param {Element} el
 * @param {string} tagName
 * @return {Array<Element>}
 * 
 * can we call it without tagName or element?
 * i have no idea
 * 
 * only descendants are looked at
 * can definitely do this recursively looking at node.children
 * Element.tagName has tagName in UPPER CASE
 * we could probably just get our own tagName to be uppercase once
 * and then just check against Element.tagName
 */
export default function getElementsByTagName(el, tagName) {
  const result = [];
  const upperCaseTagName = tagName?.toUpperCase();

  const traverse = (node) => {
    if (!node) return;

    const children = node.children;

    for (const child of children) {
      if (child.tagName === upperCaseTagName) {
        result.push(child);
      }
      traverse(child);
    }
  };

  traverse(el);
  return result;
}

## JSON.stringify()

* just know how to identify types for a value
    - `Array.isArray()` for arrays
    - typeof value === "object"
        * or for POJOs (Plain Old JavaScript Objects): `Object.getPrototypeOf(value) === Object.prototype`
    - typeof value === "string"
        * we need to do this for string because string requires double quotes around it so we have to escape it when we stringify
* you can use String() to convert directly into a string
    - String(1) => "1"
    - String(true) => "true"
    - String(null) => "null"
* Array.join(",") is quite useful if you have an array of strings
    - it will automatically take care of the last element edge case for you where you don't want a "," at the end

In [ ]:
// my solution

/**
 * @param {*} value
 * @return {string}
 * 
 * only boolean, number, null, array, and object will be present in
 * input value
 * 
 * string: typeof value === 'string'
 * boolean: [true, false].includes(value);
 * number: typeof value === 'number'
 * null: value === null
 * array: Array.isArray(value)
 * object: 
 * 
 * undefined, function(), and symbol will be omitted in an object
 * or null if found in an array
 */

/**
 * returns stringified version of primitive or false
 */
function isPrimitive (value) {
  if (value === null) return "null";
  if ([true, false].includes(value)) return `${value}`;
  if (typeof value === "string") return `\"${value}\"`;
  if (typeof value === "number") return `${value}`;

  return false;
}

export default function jsonStringify(value) {
  const type = typeof value;

  const isValuePrimitive = isPrimitive(value);

  if (isValuePrimitive) return isValuePrimitive;

  let result = "";
  // arrays
  if (Array.isArray(value)) {
    result += '[';
    value.forEach((item, index) => {
      const isValuePrimitive = isPrimitive(item);
      if (isValuePrimitive) {
        result += isValuePrimitive;
      } else {
        result += jsonStringify(item);
      }
      if (index !== (value.length - 1)) {
        result += ",";
      }
    });
    result += ']';
  } else if (type === "object") {
    result += '{';
    const entries = Object.entries(value);
    entries.forEach(([key, val], index) => {
      result += `\"${key}\":`;
      const isValuePrimitive = isPrimitive(val);
      if (isValuePrimitive) {
        result += isValuePrimitive;
      } else {
        result += jsonStringify(val);
      }
      if (index !== (entries.length - 1)) {
        result += ",";
      }
    });
    result += '}';
  }

  return result;
}

In [ ]:
// actual solution

/**
 * @param {*} value
 * @return {string}
 */
export default function jsonStringify(value) {
  if (Array.isArray(value)) {
    const arrayValues = value.map((item) => jsonStringify(item));
    return `[${arrayValues.join(',')}]`;
  }

  if (typeof value === 'object' && value !== null) {
    const objectEntries = Object.entries(value).map(
      ([key, value]) => `"${key}":${jsonStringify(value)}`,
    );
    return `{${objectEntries.join(',')}}`;
  }

  if (typeof value === 'string') {
    return `"${value}"`;
  }

  return String(value);
}
